# Does the Tourism Preferences Changes after COVID19?

## CASA0013: Foundations of Spatial Data Science

### Student Ids: ucftXXXX, ucftYYYY, etc.

## Green Space Data

In [15]:
# Import visualisation modules
import matplotlib as mpl 
%matplotlib inline 
import matplotlib.pyplot as plt 

#Import modules
import osmnx as ox
import pandas as pd
import geopandas as gpd
import numpy as np
import os

import warnings 
warnings.simplefilter(action='ignore')
ox.__version__

'1.2.2'

In [16]:
# Set up query
q1 = {
    "tourism":"camp_site",
    'leisure':'common',
    'leisure':'dog_park',
    'natural':'scrub',
    'natural':'fell',
    'landuse':'forest',
    'leisure':'garden',
    'landuse':'greenfield',
    'leisure':'golf_course',
    'landuse':'grass',
    'natural':'grassland',
    'natural':'heath',
    'landuse':'meadow',
    'natural':'moor',
    'leisure':'nature_reserve',
    'landuse':'orchard',
    'leisure':'park',
    'leisure':'pitch',
    'landuse':'recreation_ground',
    'landuse':'village_green',
    'landuse':'vineyard',
    'natural':'wood'
    }

greenspace = ox.geometries.geometries_from_place(
            'Greater London, UK',
            tags = q1,
            which_result=1)

greenspace.head(5)

geometry created_by barrier bicycle  \
element_type osmid                                                              
node         20851184    POINT (-0.33622 51.40443)        NaN     NaN     NaN   
             92273182    POINT (-0.40698 51.48916)       JOSM     NaN     NaN   
             895874399   POINT (-0.23028 51.55593)        NaN     NaN     NaN   
             920063079   POINT (-0.06894 51.56576)        NaN     NaN     NaN   
             1296074660  POINT (-0.17313 51.41807)        NaN     NaN     NaN   

                        foot source leisure                    name  \
element_type osmid                                                    
node         20851184    NaN    NaN   pitch  The Royal Tennis Court   
             92273182    NaN    NaN     NaN                     NaN   
             895874399   NaN    NaN   pitch      Kilburn Cosmos RFC   
             920063079   NaN    NaN   pitch                     NaN   
             1296074660  NaN    NaN   pitch                     NaN   

                             name:ru         sport  ... oneway construction  \
element_type osmid                                  ...                       
node         20851184    Реал-теннис   real_tennis  ...    NaN          NaN   
             92273182            NaN           NaN  ...    NaN          NaN   
             895874399           NaN         rugby  ...    NaN          NaN   
             920063079           NaN           NaN  ...    NaN          NaN   
             1296074660          NaN  table_tennis  ...    NaN          NaN   

                        proposed religion danger genus:en informal ways type  \
element_type osmid                                                             
node         20851184        NaN      NaN    NaN      NaN      NaN  NaN  NaN   
             92273182        NaN      NaN    NaN      NaN      NaN  NaN  NaN   
             895874399       NaN      NaN    NaN      NaN      NaN  NaN  NaN   
             920063079       NaN      NaN    NaN      NaN      NaN  NaN  NaN   
             1296074660      NaN      NaN    NaN      NaN      NaN  NaN  NaN   

                        network  
element_type osmid               
node         20851184       NaN  
             92273182       NaN  
             895874399      NaN  
             920063079      NaN  
             1296074660     NaN  

[5 rows x 278 columns]

In [17]:
greenspace.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 15452 entries, ('node', 20851184) to ('relation', 15268904)
Columns: 278 entries, geometry to network
dtypes: geometry(1), object(277)
memory usage: 33.4+ MB


In [18]:
path = os.path.join('data')

if not os.path.exists(path):
    print(f"Creating {path} under {os.getcwd()}")
    os.makedirs(path)
    
greenspace.to_csv(os.path.join(path,'greenspace.csv'), index=False)

## Tourism Attraction Data

In [19]:
# Set up query
q2 = {'tourism':'attraction'}

# Run query
# Jin: I change ox.pois.pois_from_place to ox.geometries.geometries_from_place, for the adjustment of osmnx version change.
# details: https://stackoverflow.com/questions/71559143/what-happened-to-the-pois-module-in-osmnx-and-what-to-use-now
tourism_attraction = ox.geometries.geometries_from_place(
            'Greater London, UK',
            tags = q2,
            which_result=1)

tourism_attraction.head(5)

historic                      name     tourism  \
element_type osmid                                                       
node         25524252   building           Blewcoat School  attraction   
             26559743        NaN                       NaN  attraction   
             252602371       NaN  London Bridge Experience  attraction   
             269236138       NaN      Little Holland House  attraction   
             293221901       NaN    Hall Place and Gardens  attraction   

                        wikidata                        wikipedia  \
element_type osmid                                                  
node         25524252   Q4926413               en:Blewcoat School   
             26559743        NaN                              NaN   
             252602371  Q7748032  en:The London Bridge Experience   
             269236138       NaN                              NaN   
             293221901  Q5642615                              NaN   

                                         geometry access barrier bicycle  \
element_type osmid                                                         
node         25524252   POINT (-0.13606 51.49830)    NaN     NaN     NaN   
             26559743   POINT (-0.14525 51.39520)    NaN     NaN     NaN   
             252602371  POINT (-0.08826 51.50639)    NaN     NaN     NaN   
             269236138  POINT (-0.17065 51.35530)    NaN     NaN     NaN   
             293221901   POINT (0.16023 51.44819)    NaN     NaN     NaN   

                       place  ... int_name source:description name:ban  \
element_type osmid            ...                                        
node         25524252    NaN  ...      NaN                NaN      NaN   
             26559743    NaN  ...      NaN                NaN      NaN   
             252602371   NaN  ...      NaN                NaN      NaN   
             269236138   NaN  ...      NaN                NaN      NaN   
             293221901   NaN  ...      NaN                NaN      NaN   

                       name:eo name:hak name:mai name:pms name:tl name:sw  \
element_type osmid                                                          
node         25524252      NaN      NaN      NaN      NaN     NaN     NaN   
             26559743      NaN      NaN      NaN      NaN     NaN     NaN   
             252602371     NaN      NaN      NaN      NaN     NaN     NaN   
             269236138     NaN      NaN      NaN      NaN     NaN     NaN   
             293221901     NaN      NaN      NaN      NaN     NaN     NaN   

                       name:xmf  
element_type osmid               
node         25524252       NaN  
             26559743       NaN  
             252602371      NaN  
             269236138      NaN  
             293221901      NaN  

[5 rows x 359 columns]

In [20]:
tourism_attraction.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 344 entries, ('node', 25524252) to ('relation', 12942436)
Columns: 359 entries, historic to name:xmf
dtypes: geometry(1), object(358)
memory usage: 976.9+ KB


In [21]:
path = os.path.join('data')

if not os.path.exists(path):
    print(f"Creating {path} under {os.getcwd()}")
    os.makedirs(path)
    
tourism_attraction.to_csv(os.path.join(path,'tourism_attraction.csv'), index=False)